In [280]:
"""
Скрипт для создания отчета по форме БРИТ
"""

'\nСкрипт для создания отчета по форме БРИТ\n'

In [281]:
import pandas as pd

In [282]:
check_cols = {'Статус_учёба','Статус_Сиротство','Статус_Уровень_здоровья'} # проверяемые колонки
dct_name_sheet = dict() # словарь где ключ это названия листа а значение датафрейм на основе которого был произведен подсчет

In [283]:
df = pd.read_excel('data/Общий файл.xlsx',dtype=str)
df.fillna('Нет статуса',inplace=True) # заполняем Наны
group_main_df = pd.DataFrame(index=list(df['Файл'].unique()))
group_main_df.index.name = 'Файл'

In [284]:
# Отбрасываем на всякий случай отчисленных
df = df[df['Статус_учёба']!= 'Отчислен']

In [285]:
df['Статус_Сиротство'].value_counts()

Нет статуса                                                       201
гособеспечение + постинтернатное сопровождение                     24
дети-сироты, находящиеся на полном государственном обеспечении     15
дети-сироты, находящиеся под опекой                                12
Name: Статус_Сиротство, dtype: int64

In [286]:
# Создаем датафрейм с сиротами
orphans_df = df[df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]

dct_name_sheet['Сироты'] = orphans_df # добавляем в словарь
orphans_group_df = orphans_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'}) # создаем базовый 
group_main_df = group_main_df.join(orphans_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Сиротство':'Дети-сироты'},inplace=True)

In [287]:
group_main_df

,Дети-сироты
Файл,
Железнодорожного транспорта,9.0
Энергоснабжения,13.0
Металлообработки,29.0
Хоринский филиал,NaN


In [288]:
# Создаем датафрейм с сиротами
invalid_df = df[df['Статус_Уровень_здоровья'].isin(['Инвалид детства','Инвалид 1,2,3, группы'])]

dct_name_sheet['Инвалиды'] = invalid_df # добавляем в словарь
invalid_group_df = invalid_df.groupby(by=['Файл']).agg({'Статус_Уровень_здоровья':'count'}) # создаем базовый 
group_main_df = group_main_df.join(invalid_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Уровень_здоровья':'Инвалиды'},inplace=True)

In [289]:
group_main_df

,Дети-сироты,Инвалиды
Файл,,
Железнодорожного транспорта,9.0,10
Энергоснабжения,13.0,10
Металлообработки,29.0,22
Хоринский филиал,NaN,3


In [290]:
# Создаем датафрейм с получателями социальной стипендии
print(df.shape)
soc_benefit_df = df[df['Статус_соц_стипендия'].isin(['да'])]
print(soc_benefit_df.shape)
dct_name_sheet['Соц. стипендия Все'] = soc_benefit_df # добавляем в словарь





(252, 53)
(30, 53)


In [291]:
# получаем малоимущих
poor_soc_benefit_df = soc_benefit_df[soc_benefit_df['Статус_Соц_положение_семьи'].isin(['Малоимущая'])]
dct_name_sheet['Соц. стипендия Малоим.'] = poor_soc_benefit_df
poor_soc_benefit_group_df = poor_soc_benefit_df.groupby(by=['Файл']).agg({'Статус_Соц_положение_семьи':'count'})
group_main_df = group_main_df.join(poor_soc_benefit_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Соц_положение_семьи':'Соц. стипендия малоимущие'},inplace=True)


In [292]:
# получаем детей сирот
orphans_soc_benefit_df = soc_benefit_df[soc_benefit_df['Статус_Сиротство'].isin(['гособеспечение + постинтернатное сопровождение','дети-сироты, находящиеся на полном государственном обеспечении',
                                                     'дети-сироты, находящиеся под опекой'])]
dct_name_sheet['Соц. стипендия Сироты'] = orphans_soc_benefit_df
orphans_soc_benefit_group_df = orphans_soc_benefit_df.groupby(by=['Файл']).agg({'Статус_Сиротство':'count'})
group_main_df = group_main_df.join(orphans_soc_benefit_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Сиротство':'Соц. стипендия дети-сироты'},inplace=True)

In [293]:
# получаем инвалидов
invalid_soc_benefit_df = soc_benefit_df[soc_benefit_df['Статус_Уровень_здоровья'].isin(['Инвалид детства','Инвалид 1,2,3, группы'])]
dct_name_sheet['Соц. стипендия Инвалиды'] = invalid_soc_benefit_df
invalid_soc_benefit_group_df = invalid_soc_benefit_df.groupby(by=['Файл']).agg({'Статус_Уровень_здоровья':'count'})
group_main_df = group_main_df.join(invalid_soc_benefit_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Уровень_здоровья':'Соц. стипендия инвалиды'},inplace=True)

In [294]:
# Создаем датафрейм с получателями бесплатного питания
print(df.shape)
eating_df = df[df['Статус_питание'].isin(['получает компенсацию за питание','питается в БРИТ'])]
print(eating_df.shape)
dct_name_sheet['Соц. стипендия Все'] = eating_df # добавляем в словарь

(252, 53)
(60, 53)


In [295]:
# получаем малоимущих
poor_eating_df = eating_df[eating_df['Статус_Соц_положение_семьи'].isin(['Малоимущая'])]
dct_name_sheet['Питание Малоим.'] = poor_eating_df
poor_eating_group_df = poor_eating_df.groupby(by=['Файл']).agg({'Статус_Соц_положение_семьи':'count'})
group_main_df = group_main_df.join(poor_eating_group_df) # добавляем в свод
group_main_df.rename(columns={'Статус_Соц_положение_семьи':'Питание малоимущие'},inplace=True)

In [296]:
group_main_df

,Дети-сироты,Инвалиды,Соц. стипендия малоимущие,Соц. стипендия дети-сироты,Соц. стипендия инвалиды,Питание малоимущие
Файл,,,,,,
Железнодорожного транспорта,9.0,10,1.0,3.0,1.0,1.0
Энергоснабжения,13.0,10,NaN,4.0,NaN,NaN
Металлообработки,29.0,22,NaN,8.0,NaN,NaN
Хоринский филиал,NaN,3,NaN,NaN,1.0,2.0
